# Global sea level budget discussion

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [8]:
# package
import os
import numpy as np
import plotly
import plotly.graph_objs as go
# script
from ncdf_io import ncdf_io
from regression import regress
from time_convert import time_convert




### File location

In [9]:
aisfile='/Users/joedhsu/Research/Rsync/Data_t/GRACE05_CSR/004_154/Antarctica/slf.t60.s300.m.deg1.scaMB_decom_multi.IVANS_IJ05_IMBIE.ptcorr.rf.200204_201410.regress.regts.agmask.-66_66.nc'
gisfile='/Users/joedhsu/Research/Rsync/Data_t/GRACE05_CSR/004_154/Greenland/slf.t60.s300.m.deg1.scaMB_decom_multi.GERUO_ICE5_COMP.ptcorr.rf.200204_201410.regress.regts.agmask.-66_66.nc'
lwmfile='/Users/joedhsu/Research/Rsync/Data_t/GRACE05_CSR/004_154/Landwater/slf.f.t60.s300.m.deg1.scaCLM4.5bgc_gict_hydt_ind_decom_multi.GERUO_ICE5_COMP.ptcorr.rf.200204_201410.regress.regts.agmask.-66_66.nc'
aodfile='/Users/joedhsu/Research/Rsync/Data_t/GRACE05_CSR/004_154/GAC/slf.t100.s0.m.deg1.rf.200204_201410.regress.regts.agmask.-66_66.nc'
glofile='/Users/joedhsu/Research/Rsync/Data_t/GRACE05_CSR/004_154/Global/slf.landf.t60.s300.m.deg1.scaCLM4.5bgc_gict_hydt_ind_decom_multi_MB_decom_multi.IVANS_IJ05_IMBIE_GERUO_ICE5_COMP.GAC.t100.ptcorr.rf.200204_201410.regress.regts.agmask.-66_66.nc'
agofile='/Users/joedhsu/Research/Rsync/Data/Argo_Scripps/RG_ArgoClim_StericH_ano_swTEOS10.200401_201612.regress.regts.-66_66.nc'
altfile='/Users/joedhsu/Research/Rsync/Data/Altimetry/AVISO/monthly_mean_dt_upd/ncfiles/data.process.2015.0217vers.CMEMS/dt_global_allsat_msla_h_giay.0.5deg.20to7.199301_201512.regress.regts.agmask.-66_66.nc'

files=[aisfile,gisfile,lwmfile,aodfile,glofile,agofile,altfile]
varname=['anoSLF_SumPacAtlInd','anoSLF_SumPacAtlInd','anoSLF_SumPacAtlInd',
         'anoSLF_SumPacAtlInd','anoSLF_SumPacAtlInd','anoSH_SumPacAtlInd',
         'anoSL_SumPacAtlInd']
varname2=['ais','gis','lwm','aod','totmass','steric','totsl']
varname2_long=['Antarctic Ice Sheet','Greenland Ice Sheet','Land Water Mass',
               'Atmospheric water vapor','Total Mass','Total Steric',
               'Total Sea Level']

In [19]:
# Dealing one file at a time 
import xarray as xr
import sys
from time_convert import tarray_month
import datetime

data=[]
for i in range(len(files)):
    ncfile=ncdf_io(files[i],verbose=0)
    ds=ncfile.read_ncdf2xarray()
    
    #old time stamp
    yeardate=np.array(ds['yeardate'])
    tdict=time_convert(yeardate).year2year_mon()
    year=tdict['year']
    month=tdict['month']

    # new time stamp
    tstamp=tarray_month([2002,1],[2016,12])
#     tstamp.to_datetime()
#     tstamp.month

    # finding the missing month and assign NaN in the ts
    if i == 0:
        gts=np.zeros([len(files),len(tstamp)])+np.float('nan')

    # creating the new xarray dataset 
    for ii in range(len(year)):
        tsig=datetime.datetime(year[ii],month[ii],15)
        ind=np.where(tstamp==tsig)[0]
        gts[i,ind]=np.round(np.array(ds[varname[i]])[ii]-np.array(ds[varname[i]])[0],decimals=1)
        
    if i == 0:   
        dict_gts={varname2[i]:(['time'],gts[i,:])}
    else:
        dict_gts[varname2[i]]=(['time'],gts[i,:])       
    
    ds_corr = xr.Dataset(dict_gts,coords={'time': tstamp})
    
    
    date_str=['%0.4i-%0.2i'%(tstamp[j].year,tstamp[j].month) for j in range(len(tstamp))]

    gplt = go.Scatter(name=varname2_long[i],
                      x=date_str,
                      y=np.array(ds_corr[varname2[i]]),
                      connectgaps=True)
    data.append(gplt)
    
    
    
    

layout = dict(
            title = 'Global Mean Sea Level',
            showlegend = True,
            calendar = "gregorian",
            xaxis= dict(title='Year'),
            yaxis= dict(title='Global Mean Sea Level (mm)')
             )
    
ds_corr.to_netcdf('./data/gmsl.nc')

# py.iplot(dict(data=data,layout=layout),filename='plot_gsl')
# # plotly.offline.plot(dict(data=data,layout=layout),filename='plot_gsl.html'\
# #                          ,include_plotlyjs=False,output_type ='div' )
plotly.offline.plot(dict(data=data,layout=layout),filename='./figures/plot_all.html')
    
    
    
    


'file:///Users/joedhsu/Github_public/data_visualization.dir/figures/plot_all.html'

### Plotting the global mean sea level contribution from mass changes 